## Dataset

O nosso dataset providencia-nos dados geográficos e contagem de casos confirmados, óbitos e recuperados do Covid-19 num periodo de  4 meses (22 de janeiro de 2020 a 5 de maio de 2020) em vários países. O objetivo será extrair informação dos dados de forma a podermos construir um modelo de regressão capaz de prever as contagens tendo por base os fatores disponiveis (localização geográfica, dia e contagem anterior). Assim sendo é necessário partir dos dados iniciais e proceder a um pré-processamento para tratar os dados e poder-se então utlizar os algoritmos de regressão que pretendemos utilizar.

### Variáveis independentes

Como foi supramencionado, pretendemos utilizar certos fatores para fundamentarmos a nossa previsão (partindo dos dados do dataset). Estes então serão as coordenadas geográficas (Latitude e Longitude), número de dias desde o inicio da previsão (ou seja, 22 de janeiro) e contagens anteriores (número de casos confirmados, óbitos e recuperados do dia anterior).

### Variáveis Dependentes

Os valores que vamos prever serão os números de casos confirmados, óbitos e recuperados, sendo então estas as nossas variáveis independentes.

## Tratamento de dados

Para podermos obter resultados fiáveis, é preciso haver um tratamento dos dados que vem do dataset.
Pegando dos dados processados anteriormente feito inicialmente, começamos por extrai-los do ficheiro para podermos manipulá-los.

In [1]:
import pandas as pd

covid_data = pd.read_csv('covid_19_clean_complete.csv')
covid_data

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.000000,65.000000,1/22/20,0,0,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0,0,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0,0,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0,0,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0,0,0
...,...,...,...,...,...,...,...,...
27451,NaN,Western Sahara,24.215500,-12.885800,5/4/20,6,0,5
27452,NaN,Sao Tome and Principe,0.186360,6.613081,5/4/20,23,3,4
27453,NaN,Yemen,15.552727,48.516388,5/4/20,12,2,0
27454,NaN,Comoros,-11.645500,43.333300,5/4/20,3,0,0


De seguida, após uma análise, verificou-se a presença de entrada respeitantes a navios que nalgum momento tiveram casos de Covid-19 e não estão portanto associados a nenhum país particular.
Além disso, consideramos que estes dados iriam criar ruído, pelo que optamos por ignorá-los e remover dos dados em análise.

In [2]:
covid_data = covid_data.drop(covid_data[covid_data['Province/State']=='Grand Princess'].index)
covid_data = covid_data.drop(covid_data[covid_data['Province/State']=='Diamond Princess'].index)
covid_data = covid_data.drop(covid_data[covid_data['Country/Region']=='Diamond Princess'].index)
covid_data = covid_data.drop(covid_data[covid_data['Country/Region']=='MS Zaandam'].index)
covid_data = covid_data.reset_index()
del covid_data['index']
covid_data

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.000000,65.000000,1/22/20,0,0,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0,0,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0,0,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0,0,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0,0,0
...,...,...,...,...,...,...,...,...
27035,NaN,Western Sahara,24.215500,-12.885800,5/4/20,6,0,5
27036,NaN,Sao Tome and Principe,0.186360,6.613081,5/4/20,23,3,4
27037,NaN,Yemen,15.552727,48.516388,5/4/20,12,2,0
27038,NaN,Comoros,-11.645500,43.333300,5/4/20,3,0,0


A maioria das entradas da coluna **Province/State** tem valores nulos, pelo que procedemos a eliminá-los. Além disso, como ter uma entrada para uma região e país não é muito relevante, optamos por agregar as duas informações numa só coluna denominada de **Local**.

In [3]:
import numpy as np

covid_data['Province/State'] = covid_data.replace(np.nan, '', regex=True)
cols = ['Province/State', 'Country/Region']
covid_data['Local'] = covid_data[cols].apply(lambda row: ' / '.join(row.values.astype(str)) if row.values[0] != '' else ''.join(row.values.astype(str)), axis=1)
del covid_data['Province/State']
del covid_data['Country/Region']
covid_data

,Lat,Long,Date,Confirmed,Deaths,Recovered,Local
48,31.8257,117.2264,1/22/20,1,0,0,Anhui / China
49,40.1824,116.4142,1/22/20,14,0,0,Beijing / China
50,30.0572,107.8740,1/22/20,6,0,0,Chongqing / China
51,26.0789,117.9874,1/22/20,1,0,0,Fujian / China
52,37.8099,101.0583,1/22/20,0,0,0,Gansu / China
...,...,...,...,...,...,...,...
26856,39.3054,117.3230,5/4/20,190,3,186,Tianjin / China
26857,31.6927,88.0924,5/4/20,1,0,1,Tibet / China
26858,41.1129,85.2401,5/4/20,76,3,73,Xinjiang / China
26859,24.9740,101.4870,5/4/20,185,2,0,Yunnan / China


De seguida, vamos converter as datas em contagem de dias desde o início do dataset (22 de janeiro de 2020)

In [4]:
with pd.ExcelWriter('covid_19_distance.xlsx') as writer:
    covid_data.to_excel(writer)

## K Nearest Neighbours
Este dataset que processamos é ideal para se usar KNN com as coordenadas geográficas para distinguir os vizinhos ao epicentro original da pandemia (Província de Wuhan, na China).
Já que pretendemos construir um modelo de regressão destes dados, vamos recorrer à ferramenta *scikit-learn* para utilizar os seus algoritmos de regressão, nos quais se encontra o **KNeighborsRegressor**, que será o escolhido para aplicar o KNN.

### KNeighborsRegressor
#### Parâmetros da pesquisa
* **n_neighbors**: número de vizinhos para usar, por defeito, 5.
* **weights**: função de peso utilizado na previsão. Valores a testar:
    * *uniform*: pesos uniformes, todos os pontos na vizinhança são pesados igualmente
    * *distance*: pesos influenciados pela distância ao ponto de pesquisa, em que pontos vizinhos mais pertos do ponto de pesquisa terão mais influência do que aqueles mais afastados
* **algorithm**: algoritmo usado para computar os nearest neighbors. Opção a usar:
    * *auto*: tenta decidir qual a melhor escolha, tendo em conta os valores passados para a função de **fit()** dentro das possíveis escolhas, *ball_tree*, *kd_tree* ou *brute*
* **metric**: definir que tipo de distância a utilizar. Métrica a testar:
    * *euclidean*: utilizar a distância euclideana (mais apropriado)
* **n_jobs**: definir nº de processos para paralelizar os trabalhos
    * *None*: não há paralelismo (usar em debug)
    * *-1*: todos os cpu's são usados

In [5]:
#data de teste: 4/5/20
#criar set de treino 
train = covid_data[(covid_data['Date']=='5/4/20') & (~covid_data['Local'].str.contains('China'))]
train

,Lat,Long,Date,Confirmed,Deaths,Recovered,Local
26780,33.000000,65.000000,5/4/20,2894,90,397,Afghanistan
26781,41.153300,20.168300,5/4/20,803,31,543,Albania
26782,28.033900,1.659600,5/4/20,4648,465,1998,Algeria
26783,42.506300,1.521800,5/4/20,750,45,499,Andorra
26784,-11.202700,17.873900,5/4/20,35,2,11,Angola
...,...,...,...,...,...,...,...
27035,24.215500,-12.885800,5/4/20,6,0,5,Western Sahara
27036,0.186360,6.613081,5/4/20,23,3,4,Sao Tome and Principe
27037,15.552727,48.516388,5/4/20,12,2,0,Yemen
27038,-11.645500,43.333300,5/4/20,3,0,0,Comoros


In [6]:
#criar set de teste
test = covid_data[(covid_data['Date']=='5/4/20') & (covid_data['Local'].str.contains('China'))]
test

,Lat,Long,Date,Confirmed,Deaths,Recovered,Local
26828,31.8257,117.2264,5/4/20,991,6,985,Anhui / China
26829,40.1824,116.4142,5/4/20,593,9,554,Beijing / China
26830,30.0572,107.8740,5/4/20,579,6,573,Chongqing / China
26831,26.0789,117.9874,5/4/20,356,1,353,Fujian / China
26832,37.8099,101.0583,5/4/20,139,2,137,Gansu / China
26833,23.3417,113.4244,5/4/20,1588,8,1571,Guangdong / China
26834,23.8298,108.7881,5/4/20,254,2,252,Guangxi / China
26835,26.8154,106.8748,5/4/20,147,2,145,Guizhou / China
26836,19.1959,109.7453,5/4/20,168,6,162,Hainan / China
26837,39.5490,116.1306,5/4/20,328,6,320,Hebei / China


In [7]:
#colunas em que vamos basear as previsões
x_columns = ['Lat','Long']
#colunas que queremos 
y_columns = ['Confirmed','Deaths','Recovered']

from sklearn.neighbors import KNeighborsRegressor
#criar o modelo e usar o número de vizinhos default, 5.
knn = KNeighborsRegressor(n_neighbors=5,weights='distance')
#aplicar a função de fit ao set de treino
knn.fit(train[x_columns],train[y_columns])
#fazer previsões do set de teste usando 
predictions = knn.predict(test[x_columns])

#tratar dos resultados
predictions = pd.DataFrame(data=predictions,columns=['Confirmed Pred.','Deaths Pred.','Recovered Pred.'])
predictions['Local'] = test['Local'].tolist()
#arredondar os valores para inteiros, com teto
predictions['Confirmed Pred.'] = predictions['Confirmed Pred.'].apply(np.ceil)
predictions['Deaths Pred.'] = predictions['Deaths Pred.'].apply(np.ceil)
predictions['Recovered Pred.'] = predictions['Recovered Pred.'].apply(np.ceil)
predictions['Confirmed Actual'] = test['Confirmed'].tolist()
predictions['Deaths Actual'] = test['Deaths'].tolist()
predictions['Recovered Actual'] = test['Recovered'].tolist()
predictions['Date'] = '5/4/20'
predictions = predictions[['Date','Local', 'Confirmed Pred.', 'Confirmed Actual','Deaths Pred.', 'Deaths Actual','Recovered Pred.','Recovered Actual']]
predictions

,Date,Local,Confirmed Pred.,Confirmed Actual,Deaths Pred.,Deaths Actual,Recovered Pred.,Recovered Actual
0,5/4/20,Anhui / China,4170.0,991,154.0,6,2579.0,985
1,5/4/20,Beijing / China,5380.0,593,154.0,9,3254.0,554
2,5/4/20,Chongqing / China,682.0,579,12.0,6,583.0,573
3,5/4/20,Fujian / China,2981.0,356,122.0,1,1603.0,353
4,5/4/20,Gansu / China,1704.0,139,31.0,2,211.0,137
5,5/4/20,Guangdong / China,1704.0,1588,100.0,8,370.0,1571
6,5/4/20,Guangxi / China,665.0,254,11.0,2,588.0,252
7,5/4/20,Guizhou / China,673.0,147,12.0,2,577.0,145
8,5/4/20,Hainan / China,614.0,168,9.0,6,542.0,162
9,5/4/20,Hebei / China,5314.0,328,152.0,6,3216.0,320


In [8]:
covid_data['Date']

0        1/22/20
1        1/22/20
2        1/22/20
3        1/22/20
4        1/22/20
          ...   
27035     5/4/20
27036     5/4/20
27037     5/4/20
27038     5/4/20
27039     5/4/20
Name: Date, Length: 27040, dtype: object